In [1]:
import logging
from shared_code.utility.spark.set_environ import set_azure_env

set_azure_env()

from shared_code.utility.scripts.reddit_collector import RedditDataCollector

logging.basicConfig(level=logging.INFO)
logging.getLogger("azure.storage").setLevel(logging.WARNING)
logging.getLogger("diffusers").setLevel(logging.WARNING)
logging.getLogger("azure.core").setLevel(logging.WARNING)

In [ ]:
import os

import pandas

import dask.dataframe as dd

from shared_code.utility.spark.set_environ import set_azure_env

set_azure_env()

from shared_code.utility.storage.table import TableAdapter

In [ ]:
from tqdm.dask import TqdmCallback

cb = TqdmCallback(desc="global")
cb.register()

from tqdm import tqdm

tqdm.pandas()
tqdm.pandas(desc="global")

In [ ]:
from tqdm import tqdm

class InnerProgressBar(tqdm):
	def __init__(self, total, desc):
		super().__init__(desc=desc)
		self.total = total
		self.current = 0

	def update_to(self):
		self.update(self.current)

In [2]:
table_name = "training"
image_out_dir = "/data/images/"

In [3]:
subreddits = "greentext+selfies+Faces+EarthPorn+CityPorn+sfwpetite+SFWNextDoorGirls+SFWRedheads"
subs = subreddits.split("+")

In [4]:
def download_subreddit_images(subreddit) -> int:
	collector: RedditDataCollector = RedditDataCollector(image_out_dir=image_out_dir, table_name=table_name)
	progress.update()
	try:
		count = collector.download_subreddit_images(subreddit)
		return count
	except Exception as e:
		logging.error(f"Error downloading images for {subreddit}: {e}")
		return 0

In [5]:
logging.getLogger("azure.storage").setLevel(logging.WARNING)

data = []

ddf = dd.from_pandas(pandas.DataFrame(subs), npartitions=len(subs))

progress = InnerProgressBar(total=len(subs), desc="global")

result = ddf.apply(lambda x: download_subreddit_images(x[0]), axis=1, meta=("count", "int64")).compute()

# for sub in subs:
# 	try:
# 		num_images_collected = collector.download_subreddit_images(sub)
# 		result = {
# 			"subreddit": sub,
# 			"count": num_images_collected
# 		}
# 		data.append(result)
# 	except Exception as e:
# 		logging.error(f"Error downloading images for {sub}: {e}")
# 		continue

display(pandas.createDataFrame(data=data))

== Starting greentext ==
All images from greentext subreddit are downloaded
== Starting selfies ==
All images from selfies subreddit are downloaded
== Starting Faces ==
All images from Faces subreddit are downloaded
== Starting EarthPorn ==
All images from EarthPorn subreddit are downloaded
== Starting CityPorn ==
All images from CityPorn subreddit are downloaded
== Starting sfwpetite ==
All images from sfwpetite subreddit are downloaded
== Starting SFWNextDoorGirls ==
All images from SFWNextDoorGirls subreddit are downloaded
== Starting SFWRedheads ==
All images from SFWRedheads subreddit are downloaded
+-----+----------------+
|count|       subreddit|
+-----+----------------+
|    0|       greentext|
|    0|         selfies|
|    0|           Faces|
|    0|       EarthPorn|
|    0|        CityPorn|
|    0|       sfwpetite|
|    0|SFWNextDoorGirls|
|    0|     SFWRedheads|
+-----+----------------+

